## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the column pax, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


## 0) Prepare booking Sample (we might need this for later)

## 1) Get familiar with data

### What if we dont want to read the whole file?

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [22]:
#DON'T DO THIS!!!
b = pd.read_csv('bookings.csv.bz2',sep='^')

In [2]:
b = pd.read_csv('bookings.csv.bz2',sep='^', nrows=9999)

In [3]:
b = pd.read_csv('bookings.sample.csv.bz2',sep='^')

In [4]:
b.shape

(9999, 38)

In [6]:
list(b.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [7]:
b.head(3)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,...,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,...,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL


In [8]:
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,...,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
1780,2013-03-25 00:00:00,1P,MX,2019eb58d520bad1cf06a1157e5a9e61,912f2a6e5288580dd1dcf463e3a7c722,d760a1a13c87ec701a1a3ff1fbef293d,2013-03-25 00:00:00,12467,0,KUL,...,KULHKG,MI,S,Y,2013-04-05 13:30:00,2013-04-05 17:17:30,1,2013,3,NULL
5703,2013-03-23 00:00:00,1A,US,96331453ea1cb8a89d366875c9e7b18c,fa6aa421a9d0c4a3cebdfe654e8a339d,ee6d4f2c163c266cfafa3d24436d9eb3,2013-02-06 00:00:00,25107,0,VLD,...,VLDATLMAD,NV,T,Y,2013-05-30 11:14:00,2013-05-31 08:38:57,1,2013,3,NULL
472,2013-03-13 00:00:00,1V,US,dbedac4352b94b29c7d9798590b6aec9,d5eb112de783180226d471a76241b0bd,f1b86abdbfff3f2049acb260dc62e4fe,2013-03-13 00:00:00,5779,0,PDX,...,ORDPDX,FK,T,Y,2013-04-20 12:08:00,2013-04-20 14:12:55,1,2013,3,NULL
2489,2013-03-10 00:00:00,1V,CA,780262d7570fba767d403645141edad9,f4da18bf2e784e4b4511e625e6a79e25,2c96a9d94becb45a2297af17f9e52618,2013-03-09 00:00:00,1987,0,YWG,...,YQMYOWYWG,KM,S,Y,2013-03-15 14:35:00,2013-03-15 19:12:06,-1,2013,3,NULL
1742,2013-03-16 00:00:00,1V,US,62e98ee8315d96c0220260bb90b8bd44,1e4ebc5f81bc9f7bfe61f3415062f663,c493e3187499f503ebc4f01e6f1a06fc,2013-02-16 00:00:00,3544,0,DEN,...,INDORDSNA,FK,T,Y,2013-03-19 17:15:00,2013-03-19 20:09:32,-1,2013,3,NULL


In [9]:
pd.set_option('display.max_columns', None)

In [10]:
b.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
4928,2013-03-05 00:00:00,1P,US,e5dc4f0da9fb8e2b20a5cda18fd18e62,a8a5bbef3d28d01c8791ffffe486aab1,ce13cbbfdf39ff7a2c2fc2e5f47980ed,2013-03-05 00:00:00,183209,0,BOS,BOS,US,CAN,CAN,CN,BOS,BOS,US,BOS,BOS,US,CAN,CAN,CN,BOSCAN,BOSCAN,CNUS,1,BOSLAXPEKCAN,MK,K,Y,2013-03-10 19:45:00,2013-03-12 11:00:41,1,2013,3,NULL
9421,2013-03-26 00:00:00,1P,SA,46283c8b2e95446f0b6bd3252a1868c8,7f8edfb30d581e83d5ba77080a0d0e4a,e382d9e949cc299807ffc0c8c88e5381,2013-03-26 00:00:00,336,4082,JED,JED,SA,CCJ,CCJ,IN,CCJ,CCJ,IN,JED,JED,SA,CCJ,CCJ,IN,CCJJED,CCJJED,INSA,1,JEDCCJ,KS,H,Y,2013-03-29 23:15:00,2013-03-30 07:21:07,1,2013,3,NULL
3219,2013-03-18 00:00:00,1A,HR,506a29d5a03ff533b99164f1b84894ab,ed6d6c35bf2e89829f44e0b944864a7c,53be3f62b57e54f8072cf6c34fc2c306,2013-01-18 00:00:00,3899,0,ZAG,ZAG,HR,IST,IST,TR,ZAG,ZAG,HR,IST,IST,TR,ZAG,ZAG,HR,ISTZAG,ISTZAG,HRTR,1,ISTZAG,EU,J,C,2013-04-01 09:05:00,2013-04-01 10:14:36,2,2013,3,ZAGH13315
8637,2013-03-08 00:00:00,1P,US,0bde7b929573cf16e0d52637cf0735d2,f187f1dfcac9833a7d7f90de67b24181,83cbaea98bfc10338c534038916c6d5c,2013-02-25 00:00:00,16025,0,DCA,WAS,US,ANC,ANC,US,DCA,WAS,US,DCA,WAS,US,ANC,ANC,US,ANCDCA,ANCWAS,USUS,0,DCAORDANC,FK,T,Y,2013-06-10 07:48:00,2013-06-10 12:55:16,2,2013,3,NULL
3635,2013-03-27 00:00:00,1A,US,ba44fc7a25848153bc0ca68260cb6429,4bd69059d2b860ca07c102938acc3181,cbbd6e277f8a60824a498c5de537ce12,2013-03-27 00:00:00,11426,0,SFO,SFO,US,BPS,BPS,BR,SFO,SFO,US,BPS,BPS,BR,SFO,SFO,US,BPSSFO,BPSSFO,BRUS,1,BPSGRUMIASFO,KK,O,Y,2013-04-16 18:20:00,2013-04-17 11:06:01,1,2013,3,NULL


In [11]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 38 columns):
act_date               9999 non-null object
source                 9999 non-null object
pos_ctry               9999 non-null object
pos_iata               9999 non-null object
pos_oid                9999 non-null object
rloc                   9999 non-null object
cre_date               9999 non-null object
duration               9999 non-null int64
distance               9999 non-null int64
dep_port               9999 non-null object
dep_city               9999 non-null object
dep_ctry               9999 non-null object
arr_port               9999 non-null object
arr_city               9999 non-null object
arr_ctry               9999 non-null object
lst_port               9999 non-null object
lst_city               9999 non-null object
lst_ctry               9999 non-null object
brd_port               9999 non-null object
brd_city               9999 non-null object
brd_ctry       

In [12]:
b.describe()

,duration,distance,intl,pax,year,month
count,9999.000000,9999.000000,9999.000000,9999.000000,9999.0,9999.0
mean,20209.746075,455.608861,0.596260,0.516152,2013.0,3.0
std,42243.233482,1738.891068,0.490671,1.783970,0.0,0.0
min,59.000000,0.000000,0.000000,-25.000000,2013.0,3.0
25%,3339.000000,0.000000,0.000000,-1.000000,2013.0,3.0
50%,7721.000000,0.000000,1.000000,1.000000,2013.0,3.0
75%,19077.000000,0.000000,1.000000,1.000000,2013.0,3.0
max,488131.000000,18481.000000,1.000000,25.000000,2013.0,3.0


In [13]:
b.groupby('rloc          ')['act_date           '].count().sort_values(ascending=False).head(10)

rloc          
ae15bcfc5aec0eb64b2c5204d08201d5    42
fb72a3899ed1cd353c5830388935e7f5    23
cd96f7b7fdb5743769053ba273c7eb5f    20
6b7878dd4ac59772e14ab4760ab45ad0    20
68aee71ee0f40aff44ed341f2c5f8627    16
2a86eac3e29c922f4a439fbc0480985b    15
58ee9c6852513816a39363a1621a0615    14
503d8dde8034a48c262b3f5764fc60ca    14
c9f19404e4f0755c40deaebd6e90ea84    13
182485c12b7e38aa6e2d24f7484c019b    12
Name: act_date           , dtype: int64

In [15]:
b[ b['rloc          ']=='ae15bcfc5aec0eb64b2c5204d08201d5'   ].sort_values('act_date           ', ascending=True)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
7745,2013-03-01 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,JNB,JNB,ZA,LAX,LAX,US,JNBLAX,JNBLAX,USZA,1,JNBFRALAX,VR,Q,Y,2013-06-09 19:05:00,2013-06-10 12:55:31,-2,2013,3,NULL
7746,2013-03-01 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,ZRH,ZRH,CH,JNB,JNB,ZA,JNBZRH,JNBZRH,CHZA,1,ZRHJNB,VI,W,Y,2013-07-22 22:45:00,2013-07-23 09:26:54,-2,2013,3,NULL
7747,2013-03-01 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,CPT,CPT,ZA,LAX,LAX,US,CPTLAX,CPTLAX,USZA,1,CPTJNBFRALAX,VR,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,2,2013,3,NULL
7748,2013-03-01 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,ZRH,ZRH,CH,CPT,CPT,ZA,CPTZRH,CPTZRH,CHZA,1,ZRHJNBCPT,VI,W,Y,2013-07-22 22:45:00,2013-07-23 14:17:39,2,2013,3,NULL
7749,2013-03-02 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,CPT,CPT,ZA,LAX,LAX,US,CPTLAX,CPTLAX,USZA,1,CPTJNBFRALAX,VR,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,-2,2013,3,NULL
7750,2013-03-02 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,ZRH,ZRH,CH,CPT,CPT,ZA,CPT,CPT,ZA,JNB,JNB,ZA,CPTJNB,CPTJNB,ZAZA,0,CPTJNB,DK,Q,Y,2013-06-09 15:10:00,2013-06-09 17:27:39,2,2013,3,NULL
7751,2013-03-04 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,ZRH,ZRH,CH,CPT,CPT,ZA,CPT,CPT,ZA,JNB,JNB,ZA,CPTJNB,CPTJNB,ZAZA,0,CPTJNB,DK,Q,Y,2013-06-09 15:10:00,2013-06-09 17:27:39,-2,2013,3,NULL
7752,2013-03-04 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,63307,0,CPT,CPT,ZA,LAX,LAX,US,CPT,CPT,ZA,CPT,CPT,ZA,LAX,LAX,US,CPTLAX,CPTLAX,USZA,1,CPTJNBFRALAX,VR,Q,Y,2013-06-09 15:10:00,2013-06-10 12:55:31,2,2013,3,NULL
7756,2013-03-07 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,ZRH,ZRH,CH,JNB,JNB,ZA,JNBZRH,JNBZRH,CHZA,1,ZRHJNB,VI,W,Y,2013-07-22 22:45:00,2013-07-23 09:26:54,2,2013,3,NULL
7755,2013-03-07 00:00:00,1P,ZA,9782cc0d8e04d24e7346699ea97d5f90,63dd6e471fa135a362bb8f9320190247,ae15bcfc5aec0eb64b2c5204d08201d5,2012-12-21 00:00:00,62781,0,JNB,JNB,ZA,LAX,LAX,US,JNB,JNB,ZA,JNB,JNB,ZA,LAX,LAX,US,JNBLAX,JNBLAX,USZA,1,JNBFRALAX,VR,Q,Y,2013-06-09 19:05:00,2013-06-10 12:55:31,2,2013,3,NULL


## Commands for obtaining data info:
   

## 2) Select the columns of interest 

In [ ]:
# arr_port, pax, year

## 3) What to do with NaN?



In [ ]:
#DROP

## Ready to code

In [30]:
import pandas as pd
pd.set_option('display.max_columns', None)
b = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999)
b = b.dropna()

In [18]:
len(b)

9999

## 4) make action plan


In [ ]:
Plan:
    1)filter 2013
    2)group by arr_port
    3)sum sobre pax
    4)ordenar sobre sum de pax
    5)top 10

In [31]:
b.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9999 entries, 0 to 9998
Data columns (total 3 columns):
arr_port    9999 non-null object
pax         9999 non-null int64
year        9999 non-null int64
dtypes: int64(2), object(1)
memory usage: 312.5+ KB


In [19]:
b=b[ b['year']==2013 ]

In [20]:
len(b)

9999

In [25]:
del b['year']

In [27]:
v

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91


In [32]:
import pandas as pd
pd.set_option('display.max_columns', None)
b = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999)
b = b.dropna()
b=b[ b['year']==2013 ]
del b['year']
b.groupby(['arr_port']).sum().sort_values('pax', ascending=False).head()

,pax
arr_port,
HKG,112
LGA,95
ORD,94
JFK,92
LAX,91


## 5) Adjust the code to work with Big data

#### but... we have to read the whole file... and with nrows we are reading ALWAYS the first N rows...


Option for reading the file:
    1) iterator=true and get_chunk
    2) define chunk_size

#### 1) iterator=true and get_chunk

In [33]:
bi = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999, iterator=True)

In [34]:
type(bi)

pandas.io.parsers.TextFileReader

In [35]:
b0=bi.get_chunk(1000)

In [37]:
b0.shape

(1000, 3)

In [38]:
b1=bi.get_chunk(8000)

In [39]:
b1.shape

(8000, 3)

In [40]:
b2=bi.get_chunk(2000)
b2.shape

(999, 3)

In [41]:
b2=bi.get_chunk(1)

StopIteration: 

#### 2) define chunk size

In [43]:
bi = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999, chunksize=3000)

In [44]:
for i, b in enumerate(bi):
    print(len(b))

3000
3000
3000
999


In [ ]:
b = b.dropna()
b=b[ b['year']==2013 ]
del b['year']
b.groupby(['arr_port']).sum()

In [51]:
import pandas as pd
pd.set_option('display.max_columns', None)

bi = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], nrows=9999, chunksize=3000)

all_chunks=pd.DataFrame()

for i, b in enumerate(bi):
    print(i, len(b))
    b = b.dropna()
    b=b[ b['year']==2013 ]
    del b['year']
    result_chunk=b.groupby(['arr_port']).sum()
    
    all_chunks=all_chunks.append(result_chunk)
    
result_all=all_chunks.reset_index().groupby(['arr_port']).sum().sort_values('pax', ascending=False).reset_index()
result_all.head(10)
#.sort_values('pax', ascending=False).head()

0 3000
1 3000
2 3000
3 999


,arr_port,pax
0,HKG,112
1,LGA,95
2,ORD,94
3,JFK,92
4,LAX,91
5,SFO,91
6,MCO,90
7,DCA,82
8,DEN,79
9,LHR,76


In [47]:
all_chunks.reset_index().shape

(1344, 2)

In [48]:
all_chunks.reset_index()

,arr_port,pax
0,ABQ,2
1,ACC,0
2,ADD,4
3,AEP,3
4,AER,1
5,AES,2
6,AGA,0
7,AGP,8
8,AJF,0
9,AKL,1


#### How do we get the all the results of all chunks at one place?


In [ ]:
Options:
    1) df.append()
    2) df.concat()
    
    3) list of small DF and then concatenate them in one DF
    4) ?

#### 2) df.concat()

 #### Concat and append should produce the same result

#### 3) list of small DF and then concatenate the list in one DF

## Pre-Bonus Solution

In [53]:
%%time
import pandas as pd
pd.set_option('display.max_columns', None)

bi = pd.read_csv('bookings.csv.bz2',sep='^', usecols=['arr_port', 'pax', 'year'], chunksize=1000000)

all_chunks=pd.DataFrame()

for i, b in enumerate(bi):
    print(i, len(b))
    b = b.dropna()
    b=b[ b['year']==2013 ]
    del b['year']
    result_chunk=b.groupby(['arr_port']).sum()
    
    all_chunks=all_chunks.append(result_chunk)
    
result_all=all_chunks.reset_index().groupby(['arr_port']).sum().sort_values('pax', ascending=False).reset_index()
result_all.head(10)
#.sort_values('pax', ascending=False).head()

0 1000000
1 1000000
2 1000000
3 1000000
4 1000000
5 1000000
6 1000000
7 1000000
8 1000000
9 1000000
10 10
CPU times: user 4min 22s, sys: 2.81 s, total: 4min 25s
Wall time: 4min 21s


In [54]:
result_all.head(10)


,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0
3,LAS,69630.0
4,JFK,66270.0
5,CDG,64490.0
6,BKK,59460.0
7,MIA,58150.0
8,SFO,58000.0
9,DXB,55590.0


In [58]:
result_all.shape

(2274, 2)

### Now we've got top airports

In [64]:
%automagic


Automagic is ON, % prefix IS NOT needed for line magics.


In [66]:
%pip install neobase

In [67]:
import neobase as nb

In [70]:
dir(nb)

['LatLng',
 'NeoBase',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'neobase']

In [71]:
help(nb)

Help on package neobase:

NAME
    neobase - # -*- coding: utf-8 -*-

PACKAGE CONTENTS
    __main__
    neobase

FILE
    /home/dsc/anaconda3/lib/python3.7/site-packages/neobase/__init__.py




In [72]:
from neobase import NeoBase

In [73]:
dir(NeoBase)

['DUPLICATES',
 'FIELDS',
 'KEY',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_distances',
 '_empty_value',
 'distance',
 'distance_between_locations',
 'find_closest_from_location',
 'find_near',
 'find_near_location',
 'find_with',
 'get',
 'get_location',
 'keys',
 'load',
 'set',
 'skip']

In [74]:
help(NeoBase)

Help on class NeoBase in module neobase.neobase:

class NeoBase(builtins.object)
 |  NeoBase(rows=None, date=None)
 |  
 |  Main structure, a wrapper around a dict, with dict-like behavior.
 |  
 |  Methods defined here:
 |  
 |  __contains__(self, key)
 |      Test if a key is in the base.
 |      
 |      :param key: the key of to be tested
 |      :returns:   a boolean
 |      
 |      >>> b = NeoBase()
 |      >>> 'AN' in b
 |      False
 |      >>> 'AGN' in b
 |      True
 |  
 |  __init__(self, rows=None, date=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Returns iterator of all keys in the base.
 |      
 |      :returns: the iterator of all keys
 |      
 |      >>> b = NeoBase()
 |      >>> sorted(b)
 |      ['AAA', 'AAA@1', 'AAB', ...
 |  
 |  __len__(self)
 |      Testing structure size.
 |      
 |      :returns: a integer
 |      
 |      >>> b = NeoBase()
 |      >>> 18000 < len(b) < 20000
 |      True
 |  
 |

In [75]:
geoDict=NeoBase()

In [76]:
dir(geoDict)

['DUPLICATES',
 'FIELDS',
 'KEY',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_build_distances',
 '_data',
 '_empty_value',
 'distance',
 'distance_between_locations',
 'find_closest_from_location',
 'find_near',
 'find_near_location',
 'find_with',
 'get',
 'get_location',
 'keys',
 'load',
 'set',
 'skip']

In [77]:
type(geoDict)

neobase.neobase.NeoBase

In [78]:
geoDict.get('LHR')

{'__dup__': set(),
 'iata_code': 'LHR',
 'name': 'London Heathrow Airport',
 'lat': '51.4775',
 'lng': '-0.461389',
 'page_rank': 0.44517643489228376,
 'country_code': 'GB',
 'country_name': 'United Kingdom',
 'continent_name': 'Europe',
 'timezone': 'Europe/London',
 'city_code_list': ['LON'],
 'city_name_list': ['London'],
 'location_type': ['A'],
 'currency': 'GBP'}

In [79]:
type(geoDict.get('LHR'))

dict

In [80]:
geoDict.get('LHR')['name']

'London Heathrow Airport'

In [81]:
result_all.head(3)

,arr_port,pax
0,LHR,88809.0
1,MCO,70930.0
2,LAX,70530.0


In [82]:
result_all.arr_port[0]

'LHR     '

### How do we remove white spaces?

In [83]:
result_all.arr_port[0].strip()

'LHR'

In [85]:
'     lkjlkjlklj      klljljlkjl     '.strip()

'lkjlkjlklj      klljljlkjl'

In [86]:
' '.join('     lkjlkjlklj      klljljlkjl     '.split())

'lkjlkjlklj klljljlkjl'

In [88]:
'     lkjlkjlklj      klljljlkjl     '.split()

['lkjlkjlklj', 'klljljlkjl']

In [89]:
result_all['arr_port']=result_all['arr_port'].str.strip()

In [90]:
result_all.arr_port[0]

'LHR'

### How do we put the name of each airport into the DF?

In [91]:
geoDict.get(result_all.arr_port[0])['name']

'London Heathrow Airport'

In [92]:
result_all['AirportName']=result_all['arr_port'].map(lambda x: geoDict.get(x)['name'])

KeyError: 'Key not found: CPQ'

In [93]:
print(len(result_all))
result_all=result_all[ result_all['arr_port'] != 'CPQ']
print(len(result_all))


2274
2273


In [94]:
result_all['AirportName']=result_all['arr_port'].map(lambda x: geoDict.get(x)['name'])

In [95]:
result_all.head()

,arr_port,pax,AirportName
0,LHR,88809.0,London Heathrow Airport
1,MCO,70930.0,Orlando International Airport
2,LAX,70530.0,Los Angeles International Airport
3,LAS,69630.0,McCarran International Airport
4,JFK,66270.0,John F. Kennedy International Airport


In [96]:
result_all=result_all.astype({'pax':int})

In [97]:
result_all.head()

,arr_port,pax,AirportName
0,LHR,88809,London Heathrow Airport
1,MCO,70930,Orlando International Airport
2,LAX,70530,Los Angeles International Airport
3,LAS,69630,McCarran International Airport
4,JFK,66270,John F. Kennedy International Airport


In [98]:
result_all.to_csv('top_airports.csv', sep='^', index=False)

In [100]:
! ls -l

total 1014708
-rw-rw-r-- 1 dsc dsc 554970628 may 31  2019 bookings.csv.bz2
-rw-rw-r-- 1 dsc dsc    535893 ene 11 09:21 bookings.sample.csv.bz2
-rw-rw-r-- 1 dsc dsc     14965 ene 11 09:56 ch_01-Empty.ipynb
-rw-rw-r-- 1 dsc dsc    199465 ene 11 13:32 ch_02-Empty.ipynb
-rw-rw-r-- 1 dsc dsc     10832 may 31  2019 ch_03-Empty.ipynb
-rw-rw-r-- 1 dsc dsc      8019 may 31  2019 ch_04-Empty.ipynb
-rw-rw-r-- 1 dsc dsc       962 may 31  2019 ch_05b-empty.ipynb
-rw-rw-r-- 1 dsc dsc     24953 may 31  2019 ch_05-empty.ipynb
-rw-rw-r-- 1 dsc dsc 483188920 may 31  2019 searches.csv.bz2
-rw-rw-r-- 1 dsc dsc     77871 ene 11 13:32 top_airports.csv


In [101]:
! head top_airports.csv

arr_port^pax^AirportName
LHR^88809^London Heathrow Airport
MCO^70930^Orlando International Airport
LAX^70530^Los Angeles International Airport
LAS^69630^McCarran International Airport
JFK^66270^John F. Kennedy International Airport
CDG^64490^Paris Charles de Gaulle Airport
BKK^59460^Suvarnabhumi Airport
MIA^58150^Miami International Airport
SFO^58000^San Francisco International Airport


## Final Solution